## Install packages

In [1]:
!pip3 install --user pandas
!pip3 install --user numpy
!pip3 install --user joblib
!pip3 install --user scipy
!pip3 install --user seaborn
!pip3 install --user tensorflow
!pip3 install --user sklearn

## Import librairies

In [2]:
import os

import pandas as pd
import numpy as np
import joblib

from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Activation, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score, classification_report

np.random.seed(seed=17)

## Aim to this notebook

In this notebook we will test the second version of preprocessed data on LSTM basic model, in order to: 
- create a usable pipeline, easy to run
- test performance 

This is a simple binary classification problem: failure or no failure. 

## Simulation Subset Selection

Use the results summary file to select a subset of simulations for model training and testing:
- randomly select 500 non-fail cases and 200 fail cases
- use simulation ids to read in the corresponding simulation data files

### Import simulation results

In [3]:
results = pd.read_csv('processed_results_summary.csv', keep_default_na=False)
results.head()

,Unnamed: 0,Simulation no#,Line,Load,Wind1,Wind2,Wind3,flag,Sequence,n_failures,t_fail,t_first_fail,dt_fail,failed component_type,failed_component_type,cascade,fail
0,0,1,Line 04 - 05,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
1,18,10,Line 26 - 28,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
2,198,100,Line 09 - 39,0.7,0.0,0.0,0.4,0,[],0,,,,,,False,False
3,1998,1000,Line 22 - 23,0.7,0.0,0.8,1.0,0,[],0,,,,,,False,False
4,19998,10000,Line 10 - 11,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False


In [4]:
results.tail()

,Unnamed: 0,Simulation no#,Line,Load,Wind1,Wind2,Wind3,flag,Sequence,n_failures,t_fail,t_first_fail,dt_fail,failed component_type,failed_component_type,cascade,fail
43060,19988,9995,Line 05 - 08,0.8,0.4,0.0,1.0,0,[],0,,,,,,False,False
43061,19990,9996,Line 05 - 06,0.8,0.4,0.0,1.0,0,[],0,,,,,,False,False
43062,19992,9997,Line 04 - 05,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False
43063,19994,9998,Line 06 - 07,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False
43064,19996,9999,Line 06 - 11,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False


In [5]:
results['fail'].value_counts()

False    35934
True      7131
Name: fail, dtype: int64

In [6]:
results['flag'].value_counts()

0    35934
1     7131
Name: flag, dtype: int64

### Get number of simulation files which are preprocessed 

Otherwise there are many missing files (not all files were preprocessed)

In [7]:
root_dir = '/output/'

preprocessed_filenames = []
for file in os.listdir(os.path.join(root_dir, '02_preprocessed_numpy')):
    file = file.split('.')[0]
    preprocessed_filenames.append(int(file))

In [8]:
len(preprocessed_filenames)

25742

#### Get the simulation ids of sims with and without failures

In [9]:
fail_ids = results[results['flag']==1]['Simulation no#'].tolist()
non_fail_ids = results[results['flag']==0]['Simulation no#'].tolist()

#### Randomly select at least 100 fail and 500 non-fail cases

In [10]:
def intersection(lst1, lst2):
    """
    Return intersection of two given lists.
    """
    return list(set(lst1) & set(lst2))

In [11]:
length_intersection = 0
while length_intersection < 100:
    fail_ids = np.random.choice(fail_ids, size=200, replace=False)
    length_intersection = len(intersection(fail_ids, preprocessed_filenames))

In [12]:
length_intersection

136

In [13]:
new_fail_ids = intersection(fail_ids, preprocessed_filenames)

In [14]:
length_intersection = 0
while length_intersection < 500:
    non_fail_ids = np.random.choice(non_fail_ids, size=1000, replace=False)
    length_intersection = len(intersection(non_fail_ids, preprocessed_filenames))

In [15]:
length_intersection

551

In [16]:
new_non_fail_ids = intersection(non_fail_ids, preprocessed_filenames)

### Load preprocessed pickle files using previously obtained ids

In [17]:
def load_numpy(directory: str, filename: str) -> np.array:
    """
    Load a numpy array from pickle format.
    """
    path = os.path.join(directory, filename)
    np_array = joblib.load(path)
    return np_array

In [18]:
root_dir = '/output/'

subset = []
for file_name in new_fail_ids:
    try:
        arr = load_numpy(os.path.join(root_dir, '02_preprocessed_numpy'), str(file_name) + '.pkl')
        sim_col = [[file_name] for x in range(arr.shape[0])]
        label_col = [[1] for x in range(arr.shape[0])]
        arr = np.append(arr, sim_col, axis=1)
        arr = np.append(arr, label_col, axis=1)
        subset.append(arr)
    except FileNotFoundError as error:
        print(f'{error} for {file_name}')

for file_name in new_non_fail_ids:
    try:
        arr = load_numpy(os.path.join(root_dir, '02_preprocessed_numpy'), str(file_name) + '.pkl')
        sim_col = [[file_name] for x in range(arr.shape[0])]
        label_col = [[0] for x in range(arr.shape[0])]
        arr = np.append(arr, sim_col, axis=1)
        arr = np.append(arr, label_col, axis=1)
        subset.append(arr)
    except FileNotFoundError as error:
        print(f'{error} for {file_name}')

In [19]:
len(subset)

687

## Generate train and test sets

- Generate evenly sized windows without shuffling the data and mixing different simulations
- Create source and target columns (the binary 'fail' column)
- Generate train and test sets with 80%-20% split

Model input formats:
- X -> (n_batch, n_time_steps, n_features)
- y -> (n_batch, 1)

In [20]:
def get_window(array, start_index, length, shift=25):
    x, y = [], []
    end_index = len(array) - length

    for i in range(start_index, end_index, shift):
        y.append(array[i:(i+length), -1][0])
        source = array[i:(i+length), 1:-2]
        x.append(source)

    return x, y

In [21]:
len(new_fail_ids), len(new_non_fail_ids)

(136, 551)

In [22]:
nb_fail_training = int(len(new_fail_ids)* (80/100))
nb_non_fail_training = int(len(new_non_fail_ids)* (80/100))

In [23]:
nb_fail_training, nb_non_fail_training

(108, 440)

In [24]:
train_ids = list(new_fail_ids[:nb_fail_training]) + list(new_non_fail_ids[:nb_non_fail_training])
test_ids = list(new_fail_ids[nb_fail_training:]) + list(new_non_fail_ids[nb_non_fail_training:])

train_X, train_y = [], []
test_X, test_y = [], []

for array in subset:
    if int(array[:, -2][0]) in train_ids:
        x, y = get_window(array, start_index=0, length=100)
        train_X.extend(x)
        train_y.extend(y)
    else:
        x, y = get_window(array, start_index=0, length=100)
        test_X.extend(x)
        test_y.extend(y)

In [25]:
# Convert train_x and test_x to 3d numpy array
train_X = np.array(train_X)
train_X = np.asarray(train_X).astype(np.float32)
train_y = np.array(train_y)

test_X = np.array(test_X)
test_X = np.asarray(test_X).astype(np.float32)
test_y = np.array(test_y)

## Model training and Evaluation

- Train a single layer LSTM model
- Evaluate on test data: accuracy, precision, recall, F1 score

In [26]:
train_X.shape

(19728, 100, 259)

In [27]:
train_y.shape

(19728,)

In [28]:
# LSTM
EPOCH_SIZE = 10
BATCH_SIZE = 20

# Number of time steps and number of features
n_samples = train_X.shape[0]
n_time_steps = train_X.shape[1]
n_feats = train_X.shape[2]

model = Sequential()
model.add(LSTM(50, input_shape = (n_time_steps, n_feats)))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Start training
model.fit(train_X, train_y, epochs=EPOCH_SIZE, batch_size=BATCH_SIZE)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                62000     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 62,051
Trainable params: 62,051
Non-trainable params: 0
_________________________________________________________________


Epoch 1/10


   20/19728 [..............................] - ETA: 25:51 - loss: 1.0160 - acc: 0.0500

...

19620/19728 [============================>.] - ETA: 0s - loss: 0.0155 - acc: 0.9950

19640/19728 [============================>.] - ETA: 0s - loss: 0.0155 - acc: 0.9950

19660/19728 [============================>.] - ETA: 0s - loss: 0.0155 - acc: 0.9950

19700/19728 [============================>.] - ETA: 0s - loss: 0.0154 - acc: 0.9950

19728/19728 [==============================] - 53s 3ms/sample - loss: 0.0154 - acc: 0.9950


In [29]:
# Predict the class of test set samples
pred_y = model.predict(test_X)
pred_y[pred_y >= 0.5] = 1
pred_y[pred_y < 0.5] = 0

# Accuracy, precision, recall, F1 score
acc = accuracy_score(test_y, pred_y)
print('Accuracy on test set: {:.3f}'.format(acc))

print(classification_report(test_y, pred_y))

Accuracy on test set: 0.985
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3996
         1.0       0.97      0.96      0.96      1008

    accuracy                           0.99      5004
   macro avg       0.98      0.97      0.98      5004
weighted avg       0.98      0.99      0.98      5004

